In [2]:
import torch
import matplotlib.pyplot as plt

In [3]:
weight=.7
bias=.3


start=0
end =1
step = 0.02
X=torch.arange(start,end,step).unsqueeze(dim=1)
y=weight*X
X[:10],y[:10],len(X),len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.0000],
         [0.0140],
         [0.0280],
         [0.0420],
         [0.0560],
         [0.0700],
         [0.0840],
         [0.0980],
         [0.1120],
         [0.1260]]),
 50,
 50)

In [4]:
train_size = int(len(X)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]  
len(X_train), len(X_test), len(y_train), len(y_test)

(40, 10, 40, 10)